In [ ]:
# default_exp checker

# Dependency Checker

> A pragmatic way to talk with pypi and find out what dependencies are out of date

In [ ]:
#hide
from nbverbose.showdoc import *

## Dependency Traversing

Sometimes, we may want to check the current installed versions of a project's basic dependencies, and further check if those dependencies are out of date. `dependency_checker` is designed around this concept, utilizing the `pipdeptree` library.

In [ ]:
#export
import json, ast, pipdeptree, sys, subprocess

In [ ]:
#export
def get_installed_dependencies(
    package_name:str, # The name of a python package
    depth_limit:int=1, # How deep to follow nested dependencies
) -> dict: # A dictionary of {package:version}
    "Recursively grabs dependencies of python package"
    pkgs = pipdeptree.get_installed_distributions(local_only=False, user_only=False)
    tree = pipdeptree.PackageDAG.from_pkgs(pkgs)
    tree = tree.filter([package_name], None)
    curr_depth=0
    def _get_deps(j, dep_dict={}, curr_depth=0):
        if curr_depth > depth_limit: return dep_dict
        if isinstance(j, list):
            for a in j:
                _get_deps(a, dep_dict, curr_depth)
        elif isinstance(j, dict):
            if 'package_name' in j.keys():
                if j['package_name'] not in dep_dict.keys():
                    dep_dict[j['package_name']] = j['installed_version']
            if 'dependencies' in j.keys():
                curr_depth += 1
                return _get_deps(j['dependencies'], dep_dict, curr_depth)
        return dep_dict
    return _get_deps(ast.literal_eval(pipdeptree.render_json_tree(tree, 4)), {})

This function operates by traversing a DAG and grabbing dependencies of projects found from it. Generally a depth of 1 is recommended, below is a quick guide to what will be returned at each depth.


**0**: A depth of zero will return just the project:

In [ ]:
deps = get_installed_dependencies('pipdeptree', depth_limit=0)

In [ ]:
assert deps == {'pipdeptree':'2.1.0'}

**1**: A depth of one will return the project and its main dependencies, such as those stated in the `requirements.txt` as well as packages such as `pip` 

In [ ]:
deps = get_installed_dependencies('pipdeptree', depth_limit=1)

In [ ]:
assert deps == {'pipdeptree':'2.1.0', 'pip':'21.2.4'}

**2+**: A depth of two or greater will return the dependencies for each of the dependencies above that layer. These allow for more fine-grained requirements

## Checking for New Versions

Given these dependencies, we can also then check for a new version to see if an upgrade is available. This is what the `check_new_version` function is designed for:

In [ ]:
#export
def check_new_version(
    package_name:str, # The name of a pip python package 
    current_version:str, # The installed version of a package, such as "1.2.3"
) -> bool: # Whether the versions are the same
    "Compares the current version with the latest version, and returns if they are different"
    latest_version = str(subprocess.run([sys.executable, '-m', 'pip', 'install', '{}==random'.format(package_name)], capture_output=True, text=True))
    latest_version = latest_version[latest_version.find('(from versions:')+15:]
    latest_version = latest_version[:latest_version.find(')')]
    latest_version = latest_version.replace(' ','').split(',')[-1]

    if latest_version == current_version:
        return False
    else:
        return True

In [ ]:
has_new_version = check_new_version('pipdeptree', '2.0.9')
assert has_new_version == True

Here we tested if `pipdeptree` has a new version available. The version we specified is one less than that of the latest release at the time of development. We got `True`, meaning a newer version is available.